<a href="https://colab.research.google.com/github/marty916/AI-Training-Colab-Notebooks/blob/main/00_Whisper_CohnVideos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 01: Download audo files using pytube

In [ ]:
!pip install tiktoken openai

In [ ]:
!pip install scrapetube

Use scrapetube to get all videoId(s) as a workaround to pytube bug in Channel

In [ ]:
import scrapetube
channel_Id = "UCggngXyZiQtd_yAZbIFWz2A" # LeadingAgile
print(channel_Id)
videos = scrapetube.get_channel(channel_Id)
print(len(videos))
#create a dictionary of metadata - videos is a generator and doesn't persist
videos_metadata_dict = {}

for video in videos:
    video_id = video['videoId']

    # Extract thumbnails
    thumbnail_list = []
    for thumb in video['thumbnail']['thumbnails']:
        thumbnail_list.append({
            'url': thumb['url'],
            'width': thumb.get('width', None),
            'height': thumb.get('height', None)
        })

    # Extract title
    title_text = ''
    if 'runs' in video['title']:
        for run in video['title']['runs']:
            title_text += run['text']

    # Extract description snippet
    description_text = ''
    if 'runs' in video['descriptionSnippet']:
        for run in video['descriptionSnippet']['runs']:
            description_text += run['text']

    videos_metadata_dict[video_id] = {
        'videoId': video_id,
        'channelId': channel_Id,
        'thumbnail': {'thumbnails': thumbnail_list},
        'title': title_text,
        'descriptionSnippet': description_text,
        'publishedTimeText': video['publishedTimeText']['simpleText'],
        'lengthText': video['lengthText']['simpleText'],
        'viewCountText': video['viewCountText']['simpleText']
    }


Install pytube from github to make sure you have the latest version. YouTube  API changes alot so this needs to stay current

In [ ]:
!pip install git+https://github.com/pytube/pytube

In [ ]:
# 1. Mount your Google Drive in Google Colab
from google.colab import drive
drive.mount('/content/drive')

# 2. Install pytube fix for Channels (backlevels pytube - not good)
# !pip install git+https://github.com/pishiko/pytube.git@42a7d8322dd7749a9e950baf6860d115bbeaedfc

from pytube import YouTube, Channel
from pytube.exceptions import RegexMatchError


In [ ]:
# 4. For each video, download the audio-only stream and save to your Google Drive
# Define the save path
save_path = "/content/drive/MyDrive/LeadingAgileYTAudio"
ytvideo_path = 'https://youtube.com/watch?v='
videos = scrapetube.get_channel("UCggngXyZiQtd_yAZbIFWz2A")
for video in videos:
    video_url = ytvideo_path + video['videoId']
    print (video_url)
    try:
        yt = YouTube(video_url)
        print (yt.video_id)
        # Get the audio-only stream
        audio_files = yt.streams.filter(only_audio=True)
        print (audio_files)


        itag = None
        for audio_file in audio_files:
          print (audio_file.mime_type)
          if audio_file.mime_type == 'audio/mp4':
            itag = audio_file.itag
            break

        if itag is None:
          print(f"No audio-only stream found for '{video_url}'")
          continue

        # Download and save the audio stream as mp3
        print (itag)
        stream =  yt.streams.get_by_itag(itag)

        stream.download(output_path=save_path, filename=f"{yt.video_id}.mp3")

    except RegexMatchError:
        print(f"RegexMatchError for '{video_url}'")
        continue
    except Exception as e:
        print(f"Error for '{video_url}': {e}")
        continue

Step 02: Install whisper and ffmpeg (this works on colab A100)

In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
!sudo apt update && sudo apt install ffmpeg

In [ ]:
import whisper
import torch
# if you have access to a GPU use it
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model = whisper.load_model("large").to(device)


In [ ]:
from pathlib import Path
# get list of MP3 audio files
save_path = "/content/drive/MyDrive/LeadingAgileYTAudio"
paths = [str(x) for x in (Path(save_path).glob('*.mp3'))]
print(len(paths))
print(paths[:5])


In [ ]:
#  debug line - just shows we're getting the videoId from the path
paths[0].split('/')[-1][:-4]


In [ ]:
from torch.cuda.memory import reset_accumulated_memory_stats
from tqdm.auto import tqdm
import json

# set window (length of text chunk) and stride
window = 1
stride = 1  # smaller stride creates overlap

data = []
results = []

with open(save_path + "transcription.jsonl", "w", encoding="utf-8") as fp:
  for i, path in enumerate(tqdm(paths)):
    _id = path.split('/')[-1][:-4]
    # transcribe to get speech-to-text
    result = model.transcribe(path)
    segments = result['segments']

    # get the video metadata
    video_metadata = videos_metadata_dict[_id]
    for j in range(0, len(segments), stride):
      j_end = min(j + window, len(segments)-1)
      text = ''.join([x["text"] for x in segments[j:j_end]])
      start = segments[j]["start"]
      end = segments[j_end]["end"]
      row_id = f"{_id}-t{segments[j]['start']}"
      meta = {
        **video_metadata,
        **{
            "id": row_id,
            "text": text.strip(),
            "start": start,
            "end": end
          }
      }
      # add results to data list
      data.append(meta)
      json.dump(meta, fp)
      fp.write('\n')


In [ ]:
print(len(segments))
